In [ ]:
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
titanic_df = pd.read_csv("./titanic_train.csv")

In [ ]:
titanic_df.head()
#passenger ID: 탑싀객 일련번호
#Survived: 생존여부(label) : 0: 사망, 1:생존
# Pclass: 선실 등급
# SibSp:  동승한 형제자매 혹은 배우자 인원수
#Parch:  동승한 부모님 또는 자녀수
# Ticket: 티켓 번호
#Embaked: 중간 정착 항구: C: Cherbourg, Q= Queenstown, S=Southhampton

In [ ]:
#일단 한 번 해보기
# target(label): Survived
# feature: 나머지 컬럼

In [ ]:
#0) 전처리

In [ ]:
# null값 처리, string 처리필요

In [ ]:
#문자열 피처를 삭제(문자열이 있으면 학습할 수 없음)



In [ ]:
X_titanic_df.info()

In [ ]:
#2) 문자열 피처 인코딩(숫자값으로 변환)

# 성별 피처를 LabelEncoding하여 숫자로 변환


In [ ]:
#Embarked 피처를 onehot Encoding하여 숫자로 변환
#Enbarked에는 Null값이 2개 있음 - 처리요망



In [ ]:
#3) NaN  결측치 채워넣기 (Age 항목 714개)



In [ ]:
X_titanic_df.info() #Age 값의 결측치 확인

In [ ]:
# Age columms을 Age_group컬럼으로 변환
# 0 ~ 5 : Baby, 6~JK: Child, 13~18: Teenager, 19~25: Student, 26~35: Young Adult, 36~60: Adult, 61~: elderly


In [ ]:
#1) 학습용, 테스트용 데이터 분리


In [ ]:
#2) 결정트리 객체 생성


In [ ]:
#3) 학습


In [ ]:
#4) 평가


In [ ]:
# 평가 점수가 낮아 전처리를 세분화해보기로 함


In [ ]:
#결정트리 최적화 - 하이퍼파라미터 튜닝 & 교차검증 -: GridSearchCV()



In [ ]:
#다른 분류 알고리듬으로 예측하기



In [ ]:
# 결정 트리 시각화

In [ ]:
%matplotlib inline

In [ ]:
#결정트리 시각화

In [ ]:
#피처와 라벨 정보를 기입하여 가독성 향상


In [ ]:
# 간단한 트리 생성

In [ ]:
#결정트리 그래프를 파일로 받아 Pdf 저장
